# DATASCI-4121
## RAQ Behavior Analysis
Alison Glazer (aglazer)

# Import Libraries

In [6]:
# Data Manipulation and Visualization
import numpy as np
import pandas as pd
import datetime as dt
import calendar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import re

# Logging
import logging

# Data Access
from data_science.yelp_data import utils
from data_science.yelp_data import query_utils
from data_science.yelp_data.redshift import clusters

from jinja2 import Template

import pickle

from IPython.display import Audio

# Settings

In [7]:
# Colors
yelp_red = '#d32323'
yelp_blue = '#0073bb'
yelp_black = '#333333'
yelp_green = '#41a700'

In [8]:
# DataFrameDisplay columns
pd.options.display.max_columns = 200
pd.options.display.max_rows = 100

In [9]:
# Logging Display
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

# Connect to Data

In [10]:
def get_cluster(name):
    """
    Connect to data
    """
    return clusters(redshift_user='aglazer')['prod-data-science'][name]


dw = get_cluster("bam-dw-v1")  # Connect to BAM Data Warehouse

# Set Up Parameters

In [11]:
start_date = '2021-01-01'
end_date = '2021-01-31'

# Queries

In [12]:
# one entry per conversation
daily_new_projects = dw.run_query(Template("""
    WITH plahperf_businesses AS (
        SELECT bcy.business_id_decid AS business_id,
            cy.alias AS alias
    FROM yelp_main_transformed.business_category_yelp bcy
    JOIN yelp_main_transformed.category_yelp cy
        ON cy.id_decid = bcy.category_yelp_id_decid
    WHERE alias IN
        ('professional', 'localservices', 'auto', 'homeservices',
                'pets', 'realestate', 'eventservices','financialservices')
    GROUP BY business_id, alias
    ),
    originating_modals AS (
        SELECT
            modal_id,
            MAX(CASE WHEN (is_source = TRUE) THEN 1 ELSE 0 END) AS originating
        FROM
            dl_sherlock.bulk_message_impression AS bmi
      WHERE DATE_TRUNC('day',bmi.session_start_time::DATETIME)
                BETWEEN '{{start_date}}' AND '{{end_date}}'
            AND DATE_TRUNC('day',bmi.dt::DATETIME)
                BETWEEN '{{start_date}}' AND '{{end_date}}'
        GROUP BY
            modal_id
    ),
    daily_projects_with_splits AS (
        SELECT
            bmd.dt AS date,
            DATE_TRUNC('week', bmd.dt::DATETIME) AS week,
            DATEPART('week', bmd.dt::DATETIME) AS week_number,
            entry_point,
            locale,
            use_case,
            originating,
            qoc_category,
            CASE
                WHEN (source='www') THEN 'www' --www
                WHEN (source='mobile_site') THEN 'mobile_site' --mobile_site
                WHEN (source='api' AND mobile_device_type='ios') THEN 'ios' --ios
                WHEN (source='api' AND mobile_device_type='android') THEN 'android'
                WHEN (source='api' AND mobile_device_type='other') THEN 'other_mobile'
                ELSE 'other'
            END AS client,
            CASE
                WHEN bmm.user_id IS NULL THEN 0
                ELSE 1
            END AS logged_in,
            project_id,
            conversation_id,
            bmd.modal_id,
            LISTAGG(DISTINCT pb.alias, ',') WITHIN GROUP (ORDER BY pb.alias ASC) AS plahperf_categories
      FROM dl_sherlock.bulk_message_delivered AS bmd
      LEFT JOIN dl_sherlock.bulk_message_modal AS bmm
        ON bmd.modal_id = bmm.modal_id
            AND bmd.session_id = bmm.session_id
            AND bmd.visitor_id = bmm.visitor_id
            AND DATE_TRUNC('day',bmm.session_start_time::DATETIME)
                        BETWEEN '{{start_date}}' AND '{{end_date}}'
                    AND DATE_TRUNC('day',bmm.dt::DATETIME)
                        BETWEEN '{{start_date}}' AND '{{end_date}}'
      LEFT JOIN originating_modals AS om
        ON bmd.modal_id = om.modal_id
      LEFT JOIN plahperf_businesses AS pb
        ON pb.business_id = bmd.business_id
      WHERE DATE_TRUNC('day',bmd.session_start_time::DATETIME)
                BETWEEN '{{start_date}}' AND '{{end_date}}'
            AND DATE_TRUNC('day',bmd.dt::DATETIME)
                BETWEEN '{{start_date}}' AND '{{end_date}}'
      GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
    )
    SELECT *
    FROM daily_projects_with_splits
    ORDER BY date,
        client,
        plahperf_categories,
        qoc_category,
        entry_point, 
        locale,
        use_case,
        originating,
        logged_in
""").render(start_date=start_date,\
    end_date=end_date))

RedshiftQueryException: (psycopg2.errors.InternalError_) This cluster has breached the allotted Spectrum quota. Please refer to stl_usage_control for details.
DETAIL:  
  -----------------------------------------------
  error:  This cluster has breached the allotted Spectrum quota. Please refer to stl_usage_control for details.
  code:      1036
  context:   
  query:     205359820
  location:  xen_execute.cpp:1556
  process:   padbmaster [pid=124746]
  -----------------------------------------------


[SQL: 
    WITH plahperf_businesses AS (
        SELECT bcy.business_id_decid AS business_id,
            cy.alias AS alias
    FROM yelp_main_transformed.business_category_yelp bcy
    JOIN yelp_main_transformed.category_yelp cy
        ON cy.id_decid = bcy.category_yelp_id_decid
    WHERE alias IN
        ('professional', 'localservices', 'auto', 'homeservices',
                'pets', 'realestate', 'eventservices','financialservices')
    GROUP BY business_id, alias
    ),
    origin

# Helpers

In [ ]:
# map entry point to page
entry_point_to_page = {'mtb_biz_widget':'biz',
    'mtb_search_action':'serp',
    'mtb_biz_sticky':'biz',
    'mtb_search_ad':'serp',
    'mtb_from_others':'biz',
    'mtb_service_offerings':'biz',
    'web_view_cta':'biz',
    'search_header_banner':'serp',
    'mobile_search_banner':'serp',
    'mtb_biz_objective_targeting_sticky_cta':'biz',
    'mtb_biz_after_call':'biz',
    'mtb_landing_page':'seo',
    'third_party_deep_link':'unknown',
    'sem_landing':'sem',
    'serp_questions_on_raq_entry_widget':'serp',
    'search_separator_banner':'serp',
    'mtb_portfolio_sticky':'biz',
    'entry':'unknown',
    'article':'seo',
    'mtb_seo_page':'seo',
    'verified_license':'unknown'}

# Analysis

In [ ]:
daily_new_projects['entry_page'] = daily_new_projects['entry_point'].map(entry_point_to_page)

In [ ]:
def plot_cut(df,column,subtitle=None):
    agg_df_modals = df.groupby(column).modal_id.nunique().reset_index().sort_values('modal_id', ascending=False)
    # agg_df_modals['percent'] = agg_df_modals.modal_id.transform(lambda x: 100 * x / x.sum())
    plt.bar(np.arange(len(agg_df_modals)), agg_df_modals['modal_id'], tick_label = agg_df_modals[column])
    plt.title(column)
    return

In [ ]:
plot_cut(daily_new_projects, 'entry_page')